In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test_stage1.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       1920 non-null   int64 
 1   comment  1920 non-null   object
 2   label    1920 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 45.1+ KB


In [3]:
df_train['label'].value_counts()

1    960
0    960
Name: label, dtype: int64

In [4]:
df_train.head()

,ID,comment,label
0,1,انا اوصي من هذا المنبر الكل للتوجه الى مراكز ا...,1
1,2,\nهناك الكثير لا يفهم المقصود كورونا ليست صعبة...,1
2,3,الحمد لله ارقام مقبولة مقارنة بدول المنطقة لول...,1
3,4,انا شخصيا أؤيد ما فرضته السلطات من ضرورة الإدل...,1
4,5,\nنفس الشئ في مدينة برشيد مراكز التلقيح مغلقة ...,1


In [5]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def normalize_characters(text):
    return text.replace('ة', 'ه').replace('ى', 'ي')
def remove_stop_words(text):
    stop_words = set(['ال', 'و', 'في', 'على', 'من', 'إلى', 'عن', 'مع', 'ب', 'ل', 'ها', 'هو', 'هي', 'هم', 'هن'])
    return " ".join(w for w in text.split() if w not in stop_words)

def clean_text(text):
    text = normalize_characters(text)  # Normalize characters
    text = remove_stop_words(text)  # Remove stop words
    text.replace('\n', ' ')
    text = word_tokenize(text)


    return text


def clean_text1(text):
    text = normalize_characters(text)  # Normalize characters
    text = remove_stop_words(text)  # Remove stop words
    text.replace('\n', ' ')
    # text = word_tokenize(text)


    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
df_train["clean_comment"] = df_train["comment"].apply(clean_text)
df_train["clean_comment"].head()

0    [انا, اوصي, هذا, المنبر, الكل, للتوجه, الي, مر...
1    [هناك, الكثير, لا, يفهم, المقصود, كورونا, ليست...
2    [الحمد, لله, ارقام, مقبوله, مقارنه, بدول, المن...
3    [انا, شخصيا, أؤيد, ما, فرضته, السلطات, ضروره, ...
4    [نفس, الشئ, مدينه, برشيد, مراكز, التلقيح, مغلق...
Name: clean_comment, dtype: object

In [7]:
df_train["comment"] = df_train["comment"].apply(clean_text1)
df_train["comment"].head()

0    انا اوصي هذا المنبر الكل للتوجه الي مراكز التل...
1    هناك الكثير لا يفهم المقصود كورونا ليست صعبه ع...
2    الحمد لله ارقام مقبوله مقارنه بدول المنطقه لول...
3    انا شخصيا أؤيد ما فرضته السلطات ضروره الإدلاء ...
4    نفس الشئ مدينه برشيد مراكز التلقيح مغلقه إلي غ...
Name: comment, dtype: object

In [8]:
df_train['label_for_ftext'] = '__label__' + df_train['label'].astype(str)
df_train.head(5)

,ID,comment,label,clean_comment,label_for_ftext
0,1,انا اوصي هذا المنبر الكل للتوجه الي مراكز التل...,1,"[انا, اوصي, هذا, المنبر, الكل, للتوجه, الي, مر...",__label__1
1,2,هناك الكثير لا يفهم المقصود كورونا ليست صعبه ع...,1,"[هناك, الكثير, لا, يفهم, المقصود, كورونا, ليست...",__label__1
2,3,الحمد لله ارقام مقبوله مقارنه بدول المنطقه لول...,1,"[الحمد, لله, ارقام, مقبوله, مقارنه, بدول, المن...",__label__1
3,4,انا شخصيا أؤيد ما فرضته السلطات ضروره الإدلاء ...,1,"[انا, شخصيا, أؤيد, ما, فرضته, السلطات, ضروره, ...",__label__1
4,5,نفس الشئ مدينه برشيد مراكز التلقيح مغلقه إلي غ...,1,"[نفس, الشئ, مدينه, برشيد, مراكز, التلقيح, مغلق...",__label__1


In [9]:
df_train['label_comment'] = df_train['label_for_ftext'] + ' ' + df_train['comment']


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_train, test_size=0.2)

In [11]:
train.to_csv("comment.train", columns=["label_comment"], index=False, header=False)
test.to_csv("comment.test", columns=["label_comment"], index=False, header=False)

In [12]:
!pip install gensim

In [13]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.bin.gz

--2024-01-08 15:23:13--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.244.202.103, 18.244.202.25, 18.244.202.62, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.244.202.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4500982519 (4.2G) [application/octet-stream]
Saving to: ‘cc.ar.300.bin.gz’

cc.ar.300.bin.gz    100%[===================>]   4.19G  60.3MB/s    in 88s     

2024-01-08 15:24:41 (49.0 MB/s) - ‘cc.ar.300.bin.gz’ saved [4500982519/4500982519]



In [14]:
!gunzip cc.ar.300.bin.gz
!pip install fastText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=ba0a457477145780d66e24a7ddc9a8740aca367456936f92a39ed310155446fa
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fastText


In [15]:
import fasttext
import fasttext.util

model = fasttext.load_model('cc.ar.300.bin')
model.get_dimension()

300

In [16]:
model = fasttext.train_supervised("/content/comment.train", epoch=1000)

In [17]:
model.test("/content/comment.test")

(365, 0.7753424657534247, 0.7753424657534247)

In [18]:
df_train["embeddings"] = df_train["comment"].apply(lambda x: model.get_sentence_vector(x))

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               1920 non-null   int64 
 1   comment          1920 non-null   object
 2   label            1920 non-null   int64 
 3   clean_comment    1920 non-null   object
 4   label_for_ftext  1920 non-null   object
 5   label_comment    1920 non-null   object
 6   embeddings       1920 non-null   object
dtypes: int64(2), object(5)
memory usage: 105.1+ KB


In [20]:
from sklearn.model_selection import train_test_split
embeddings = df_train["embeddings"].tolist()

X_train, X_test, y_train, y_test = train_test_split(embeddings, df_train['label'], test_size=0.2, random_state=42)

In [21]:
import numpy as np
from sklearn.svm import SVC

model_SVC = SVC(kernel='rbf')
model_SVC.fit(X_train, y_train)

accuracy = model_SVC.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9557291666666666


In [22]:
import numpy as np
from sklearn.linear_model import LogisticRegression

model_log = LogisticRegression(random_state=42)

model_log.fit(X_train, y_train)

accuracy = model_log.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9557291666666666


In [23]:
import xgboost as xgb

# Create the model
model_xgb = xgb.XGBClassifier(random_state=42)  # Set random_state for reproducibility

# Fit the model
model_xgb.fit(X_train, y_train)

# Evaluate the model
accuracy = model_xgb.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.953125


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create the Random Forest model
model_rf = RandomForestClassifier(n_estimators=100,random_state=42)  # Set random_state for reproducibility

# Fit the Random Forest model
model_rf.fit(X_train, y_train)

# Make predictions
y_pred = model_rf.predict(X_test)

# Evaluate the Random Forest model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.953125


In [28]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

model_DL = Sequential()

model_DL.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model_DL.add(Dropout(0.3))

model_DL.add(Dense(128, activation='relu'))
model_DL.add(Dropout(0.2))

model_DL.add(Dense(64, activation='relu'))
model_DL.add(Dropout(0.1))

model_DL.add(Dense(1, activation='sigmoid'))

model_DL.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_DL.fit(X_train, y_train, epochs=200, batch_size=100)

# Evaluate the model_DL
accuracy = model_DL.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)


Epoch 1/200
16/16 [==============================] - 1s 7ms/step - loss: 0.5983 - accuracy: 0.9102
Epoch 2/200
16/16 [==============================] - 0s 6ms/step - loss: 0.2586 - accuracy: 0.9427
Epoch 3/200
16/16 [==============================] - 0s 7ms/step - loss: 0.1770 - accuracy: 0.9466
Epoch 4/200
16/16 [==============================] - 0s 6ms/step - loss: 0.1666 - accuracy: 0.9447
Epoch 5/200
16/16 [==============================] - 0s 6ms/step - loss: 0.1652 - accuracy: 0.9421
Epoch 6/200
16/16 [==============================] - 0s 6ms/step - loss: 0.1677 - accuracy: 0.9388
Epoch 7/200
16/16 [==============================] - 0s 6ms/step - loss: 0.1651 - accuracy: 0.9414
Epoch 8/200
16/16 [==============================] - 0s 6ms/step - loss: 0.1592 - accuracy: 0.9466
Epoch 9/200
16/16 [==============================] - 0s 6ms/step - loss: 0.1618 - accuracy: 0.9460
Epoch 10/200
16/16 [==============================] - 0s 6ms/step - loss: 0.1609 - accuracy: 0.9434
Epoch 11/

#generate labels for the test stage data set

In [25]:
import pandas as pd
df_test_stage1= pd.read_csv('/content/test_stage1.csv')
df_test_stage1.head()

df_test_stage1["comment"] = df_test_stage1["comment"].apply(clean_text1)
df_test_stage1["comment"].head()
df_test_stage1["embeddings"] = df_train["comment"].apply(lambda x: model.get_sentence_vector(x))
labels=model_SVC.predict(list(df_test_stage1["embeddings"]))

df_test_stage1['label'] = labels

df_test_stage1[['ID', 'label']].to_csv('/content/test_prediction.csv', index=False)

df_test_stage1[['ID', 'comment','label']].to_csv('/content/test_prediction_allcolumns.csv', index=False)